In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Ensemble


Ensemble 기법을 활용한 분류기 (이번 강의 노트에서 배우게 되는)
1. Voting based classifer-> various toos for maching learning could be used.
2. Bagging
3. Random Forest

1. Voting based classifier (투표 기반 분류기)

Hard voting -0,1
Soft voting classifier-label
general voting 
random classifier could be the probability of face or rear would be same


Hard voting or Soft voting classifi

more the weak learner summed stronger the power of learner could be 


# loading data

In [1]:
import sklearn.datasets as datasets

x,y = datasets.make_moons(n_samples=1000, noise=0.3, random_state=42)

In [3]:
x[:10]

array([[-0.17186341,  0.59624885],
       [ 1.25328273, -0.26541353],
       [ 0.72322405,  0.2319425 ],
       [-0.06519764, -0.65519443],
       [-0.79949325,  0.55293543],
       [ 0.71131439,  0.07365753],
       [ 0.26252452, -0.17853072],
       [-0.17500737,  0.5947448 ],
       [ 1.54052117, -0.36735281],
       [ 0.81833812,  1.06585092]])

`학습 / 테스트셋 구분

In [7]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, random_state=42)

Hard voting : Baseearner의 결과값이 클래스이며, major class를 사용하여 최종 결과를 도출한다
* Base learner

* LogisticRegression
* RandomForestClassifier
* SVC


In [8]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Base learner들
log_clf = LogisticRegression(solver="liblinear", random_state=42)
# solver is equoation for optimazation
dt_7_clf = DecisionTreeClassifier(max_depth=7, random_state=42)
dt_10_clf = DecisionTreeClassifier(max_depth=10, random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('dt_7', dt_7_clf), ('dt_10', dt_10_clf)], voting='hard'

SyntaxError: unexpected EOF while parsing (<ipython-input-8-8dee09e855a5>, line 13)

In [9]:
# 개별 학습기의 성능 (랜덤포레스트, 로지스틱, SVM, 다수결 기반 분류기)
for clf in (log_clf, dt_7_clf, dt_10_clf, voting_clf):
    clf.fit(train_x, train_y)
    test_pred = clf.predict(test_x)
    print(clf.__class__.__name__, accuracy_score(test_y, test_pred))

NameError: name 'log_clf' is not defined

In [ ]:
log_clf = LogisticRegression(solver="liblinear", random_state=42)
dt_7_clf = DecisionTreeClassifier(max_depth=7, random_state=42)
dt_10_clf = DecisionTreeClassifier(max_depth=10, random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('dt_7', dt_7_clf), ('dt_10', dt_10_clf)], voting='soft'
)

# 2. Bagging and Pasting
* Bagging (Boostrap Aggregating)
* 훈련 세트에서 중복을 허용하여 샘플링을 하고, 그 훈련 세트에서 1개의 학습 알고리즘을 훈련 시킨 결과를 조합하는 기법
* 모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측을 생성
분류의 경우는 통계적 최빈값(즉, hard voting classifier 처럼 가장 많은 예측 결과)을 사용
회귀의 경우는 평균을 사용
개별 예측기는 원본 훈련 세트로 훈련시킨 것보다 훨씬 크게 편향되어 있지만 앙상블의 결과 편향과 분산이 모두 감소 (일반적으로 앙상블 기법에서는 편향은 비슷하지만, 분산은 줄어듬)
병렬로 학습이 가능
Pasting
훈련 세트에서 중복을 허용하지 않고 샘플링을 하고, 그 훈련 세트에서 1개의 학습 알고리즘을 훈련 시킨 결과를 조합하는 기법

In [12]:
import multiprocessing

In [13]:
#Bagging 학습
from sklearn.ensemble import BaggingClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1
)
# -> base learner could be "decisiontreeclassifier"
# n_jobs means all  threads i  have could be used.
# 

bag_clf.fit(train_x, train_y)
pred_y = bag_clf.predict(test_x)

NameError: name 'DecisionTreeClassifier' is not defined

In [ ]:
accuracy_score(test_y, pred_y)

In [ ]:
tree = DecisionTreeClassifier(random_state=42)
tree.fit(train_x, train_y)
pred_y_tree = tree.predict(test_x)

In [ ]:
accuracy_score(test_y, pred_y_tree)

OOB (out-of-bag)
배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링되고 어떤 것은 전혀 선택되지 않을 가능성이 존재한다. BaggingClasiifer는 기본값으로 중복을 허용하여(with replacement) 훈련 세크의 크기 만큼인 m개 샘플을 선택한다. 이는 평균적으로 각 예측기에 훈련 샘플의 63% 정도만 샘플링된다는 것을 의미한다.

m개의 샘플에서 무작위로 하나를 추출할 때 선택되지 않을 확률이 1−1m이고 이를 m번 반복했을 때도 선택되지 않을 확률은 (1−1m)m. 로그와 로피탈 정리를 이용하면 e−1과 같아진다. 따라서 샘플링될 확률은 1−e−1이 된다.
여기서 선택되지 않은 훈련 샘플의 나머지 37%를 oob 샘플이라고 한다. 예측기마다 남겨진 37%는 모두 다르다.

따라서 예측기가 훈련되는 동안에는 oob 샘플을 사용하지 않으므로 검증 세트나 교차 검증을 사용하지 않고 oob 샘플을 사용하여 평가할 수 있다. 앙상블의 평가는 각 예측기의 oob 평가를 평균하여 얻는다.

Sklearn에서는 oob_score_를 이용한다.-> means oob sample could be use as validation data set.

3. Random forest
랜덤 포레스트는 일반적으로 bagging 방법(또는 pasting)을 적용한 결정 트리의 앙상블이다.
랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 *전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입한다.*

트리를 더욱 다양하게 생성 (트리의 의존성을 낮추고, 다양성을 증가)
편향을 손해 보는 대신 분산을 낮추어 전체적으로 더 훌륭한 모델을 생성

Sklearn에서 제공하는 RandomForestClassifier을 사용하면 된다.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 16개의 리프 노드를 갖는 500개의 트리로 이루어진 랜덤 포레스트를 여러 CPU 코어에서 훈련
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42)
rnd_clf.fit(train_x, train_y)

In [ ]:
pred_rf_y = rnd_clf.predict(test_x)
accuracy_score(test_y, pred_rf_y)

Sklearn에서 제공하는 BaggingClassifier 이용하여 Random forest를 구현하는 방법도 있다. splitter="random"을 설정하면 됩니다.
-> BaggingClassifier = randomforest


In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(splitter="random", max_leaf_nodes=16, random_state=42),
    n_estimators=500, 
    max_samples=1.0, 
    bootstrap=True, 
    n_jobs=-1, 
    random_state=42)

bag_clf.fit(train_x, train_y)

Extra tree
Extremely Randomized Trees (Extra Trees) : 랜덤 포레스트에서 트리를 만들 때 각 노드는 무작위로 특성의 서브셋을 만들어 분할에 사용한다. 특성의 서브셋에 대하여, 각 특성 모두에 대해서 계산을 하고 분할에 최적화된 임계값을 찾는다(greedy search). 트리를 더욱 무작위로 만들기 위해, 보통의 결정 트리처럼 후보 특성의 greedy search를 하여 최적의 임곗값을 찾는 대신 후보 특성을 사용해 무작위로 분할한 다음 그중에서 최상의 분할을 선택한다. 따라서 일반적인 랜덤 포레스트보다 엑스트라 트리가 훨씬 빠르다.
RandomForestClassifier vs ExtraTreesClassifier : 어떤 것이 더 좋을지는 속단하기 어렵다. 일반적으로 둘 다 시도해보고 교차 검증으로 비교해보는 것이 유일한 방법이다. 그리고 그리드 탐색으로 하이퍼파라미터 튜닝을 한다.

-> 무작위로 뽑은 데이터에서 지니계수를 구하는 것이 아니라 익스트림은 지니계수를 구하지도 않고 무작위로 뽑은 데이터를 그냥 쓰는 것임.

decision tree belongs to white box
random forest belongs to black box

feature importance can be important

통계적인 유의성이라고 절대 말하면 안됨! (평균이니까)


In [ ]:
rnd_clf.feature_importances_

Iris를 random forest를 이용하여 분류하고 난 후, feature 중요도를 구해보자.

In [ ]:
# Iris data set

from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=42)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
rf_feat_importance = rnd_clf.feature_importances_
indices = np.argsort(rf_feat_importance)

plt.title('RF feature importnace')
plt.barh(range(len(indices)), rf_feat_importance[indices], color='b', align='center')
plt.yticks(range(len(indices)), np.array(iris["feature_names"])[indices])
plt.xlabel('Relative Importance')